In [15]:
import pandas as pd
import numpy as np
import ta

# Single CandleStick Patterns

In [30]:
def Check_Marubozu(Shadow_u,Shadow_l,Open,Close):
    State=0
    if Shadow_u<0.1 and Shadow_l<0.1:
        if Open<Close:
            State=1
        else:
            State=-1
    return State

In [31]:
def Check_doji(Shadow_u,Shadow_l):
    if Shadow_u >4 and Shadow_l>4:
        return "Doji Yes"
    else:
        return "Doji No"

In [32]:
def Check_SCP(Open,High,Low,Close,A):
    if Open<Close:
        Shadow_u=(High-Close)/(Close-Open)
        Shadow_l=(Open-Low)/(Close-Open)
    else :
        Shadow_u=(High-Open)/(Open-Close+1)
        Shadow_l=(Close-Low)/(Open-Close+1)
    A.append(Check_Marubozu(Shadow_u,Shadow_l,Open,Close))
    
    A.append(Check_doji(Shadow_u,Shadow_l))
    return A

# Indicator Functions

In [16]:
def Check_RSI(Open,High,Low,Close,UpperBound,LowerBound):
    
    RSI=ta.momentum.RSIIndicator(Close).rsi()
    Rsi_Check=RSI.iloc[-4:-1]
    State=0
    for i in Rsi_Check:
        if i >= UpperBound:
            State=-1
        elif i <= LowerBound:
            State=+1
    
    return State

In [17]:
def Check_MACD_Cross(Open,High,Low,Close):
    MACD=ta.trend.MACD(Close).macd()
    State=0
    if MACD.iloc[-1]>=0:
        if MACD.iloc[-2]<0:
            State= 1
    elif MACD.iloc[-1]<=0:
        if MACD.iloc[-2]>0:
            State= -1

    return State

In [18]:
def Check_MACD_Invert(Open,High,Low,Close):
    MACD=ta.trend.MACD(Close).macd()
    MACD=MACD[-7:].tolist()
    State=0
    if MACD[-1]!=max(MACD) and MACD[-1]!=min(MACD):
        if MACD[-2]-MACD[-1]>0:
            State=1
        else :
            State= -1
    return State

In [40]:
def Check_BollingerBands(Open,High,Low,Close):
    State = 0
    State = -1*max(ta.volatility.bollinger_hband_indicator(Close).iloc[-1],0)
   
    if State==0:
        State=max(ta.volatility.bollinger_lband_indicator(Close).iloc[-1],0)

    return int(State)

In [20]:
def Check_Volume(Volume):
    VEMA=ta.trend.ema_indicator(Volume,20)
    if Volume.iloc[-1]>VEMA.iloc[-1]:
        return "Volume Yes"
    else :
        return "Volume No"

In [26]:
def Check_Indicators(Open,High,Low,Close,Volume,A,):
    #Check RSI
    A.append(Check_RSI(Open,High,Low,Close,80,20))
    A.append(Check_RSI(Open,High,Low,Close,70,30))
    
    #Check MACD
    A.append(Check_MACD_Cross(Open,High,Low,Close))
    A.append(Check_MACD_Invert(Open,High,Low,Close))
    #Check Boollinger Bands
    A.append(Check_BollingerBands(Open,High,Low,Close))
    
    A.append(Check_Volume(Volume))
    
    return A

# Main Function

In [41]:
Watchlist_tickers=pd.read_csv('Data/Watchlist_yahoo_tickers.csv')
Watchlist_list=Watchlist_tickers["Tickers"]
TechnicalAnalysis=pd.DataFrame([],columns=['Name','Marubozu','Doji','RSI Low Risk','RSI High Risk','MACD Cross','MACD Invert','Bollinger Bands','Volume','Total'])
for i in Watchlist_list:
    A=[]
    A.append(i)
    try:
        df = pd.read_excel('Data/Historical_data.xlsx',sheet_name=i)
    except:
        print(f'{i} not Found')
    
    #Checking for Single Candlestick Patterns
    Open=df.iloc[-1]["Open"]
    High=df.iloc[-1]["High"]
    Low=df.iloc[-1]["Low"]
    Close=df.iloc[-1]["Close"]
    Check_SCP(Open,High,Low,Close,A)
    
    #Checking with indicators
    Open=df["Open"]
    High=df["High"]
    Low=df["Low"]
    Close=df["Close"]
    Volume=df["Volume"]
    Check_Indicators(Open,High,Low,Close,Volume,A)
    #Finding Conclusions
    sum=0
    multiplier=1
    for i in A:
        if i==1 or i==-1 :
            sum+=i
        if i=='Volume Yes' :
            multiplier*=2
        if i=='Volume No' :
            multiplier*=0.5
        if i=='Doji Yes':
            multiplier*=2
    sum=sum*multiplier
    A.append(sum)
    #Updating the output dataframe
    a_series = pd.Series(A, index = TechnicalAnalysis.columns)
    TechnicalAnalysis = TechnicalAnalysis.append(a_series, ignore_index=True)

print(TechnicalAnalysis)

             Name Marubozu     Doji RSI Low Risk RSI High Risk MACD Cross  \
0   ADANIPORTS.NS        0  Doji No            0            -1          0   
1   ASIANPAINT.NS        0  Doji No           -1            -1          0   
2     AXISBANK.NS        0  Doji No            0            -1          0   
3   BAJAJ-AUTO.NS        0  Doji No            0             0          0   
4   BAJFINANCE.NS        0  Doji No            0            -1          0   
..            ...      ...      ...          ...           ...        ...   
94        SAIL.NS        0  Doji No            0            -1          0   
95  TORNTPHARM.NS        0  Doji No            0             0          0   
96  MCDOWELL-N.NS        0  Doji No            0             0          0   
97        VEDL.NS        0  Doji No            0             0          0   
98     YESBANK.NS        0  Doji No            0             0          0   

   MACD Invert Bollinger Bands      Volume Total  
0            0          

In [39]:
#export to Excel
TechnicalAnalysis.to_excel("Data\LongShortOpportunites.xlsx",sheet_name='Equities')